# import libraries

In [742]:
import ee
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Authentication and Initialization  -
Before you can make requests to Earth Engine through a client library, you must authenticate

In [744]:
ee.Authenticate()

True

In [745]:
ee.Initialize()

# Get statistics of Rectangels(regions) around the specific points

In [747]:
#dowenload the image collection and filter it with date and get the median image
dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2022-09-01', '2022-09-30').median()  

In [748]:
# Define the points (coordinates)
points = [
    [30.488472, 31.173472],
    [30.524722, 31.151861],
    [30.566028, 31.130972],
    [30.647556, 31.077806],
    [30.734028, 31.028500],
    [30.807556, 31.013361],
    [30.790639, 30.965194],
    [30.801278, 30.913278],
    [30.815306, 30.904389],
    [30.848333, 30.901222],
    [30.876917, 30.873500],
    [30.899056, 30.856250],
    [30.930333, 30.835639],
    [30.961583, 30.783083],
    [31.003111, 30.759333],
    [31.043556, 30.735694],
    [31.085667, 30.727861],
    [31.101222, 30.683639],
    [31.143611, 30.645361],
    [31.182500, 30.600361],
    [31.207500,30.569444],
    [31.224444,30.531583],
    [31.264361,30.518389],
    [31.311250,30.515528],
    [31.344417,30.482861],
    [30.851444,31.030083],
    [30.894389,31.024361],
    [30.893667,31.024667],
    [30.931611,30.990083],
    [30.980056,30.970417],
    [31.031639,30.961556],
    [31.085750,30.965417],
    [31.142111,30.968861],
    [31.214722,30.984111],
    [31.268806,30.990528],
    [31.261778,31.049611],
    [31.278194,31.093417],
    [31.328944,31.132139],
    [31.399000,31.125000],
    [31.471750,31.126861],
    [31.526444,31.140306],
    [31.565361,31.081889],
    [31.586472,31.052500],
    [31.412167,30.451000],
    [31.340417,30.366278],
    [31.205000,30.091444],
    [31.167278,30.069167],
    [31.128167,30.102194],
    [31.120861,30.164528],
    [31.108250,30.211944],
    [31.086417,30.294417],
    [31.069194,30.359806],
    [31.013250,30.473583],
]

In [749]:
# Create a feature collection from the points
features = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(coord)) for coord in points])

In [750]:
# Define the size of the rectangle in meters (1 km x 1 km)
size = 7000  # Size in meters

In [751]:
# Create a function to create a rectangle around a point
def create_rectangle(point):
    lat = point[1]
    lon = point[0]
    
    # Calculate the corners of the rectangle
    topLeft = [
        lon - (size / 20037508.34 * 180 / 3.141592653589793),
        lat + (size / 20037508.34 * 180 / 3.141592653589793)
    ]
    
    bottomRight = [
        lon + (size / 20037508.34 * 180 / 3.141592653589793),
        lat - (size / 20037508.34 * 180 / 3.141592653589793)
    ]
    
    # Return the rectangle geometry
    return ee.Geometry.Rectangle(
        topLeft[0], bottomRight[1],
        bottomRight[0], topLeft[1]
    )

In [752]:
# Create a feature collection of rectangles
rectangles = ee.FeatureCollection([ee.Feature(create_rectangle(pt)) for pt in points])

In [753]:
# Function to get statistics for each rectangel
def get_statistics(feature):
    point = feature.geometry()
    # Calculate the median for each band in the median image
    stats = dataset.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=point,
        scale=30  # resolution in meters
    )
    return feature.set(stats)

In [754]:
# Apply the function to each rectangel in the features collection
stats_features = rectangles.map(get_statistics)

In [ ]:
# Extract features and their statistics from server-side
features = stats_features.getInfo().get('features', [])

# prepare the table data to store the statistics results

In [755]:
# Initialize a data table as a dictionary
data_table = {
    'Point Longitude': [],
    'Point Latitude': [],
    'B1': [],
    'B2': [],
    'B3': [],
    'B4': [],
    'B5': [],
    'B6': [],
    'B7': [],
    'B8': [],
    'B8A': [],
    'B9': [],
    'B11':[],
    'B12':[]
}
# Loop through features and populate the data table
for feature in features:
    properties = feature['properties']
    coords = feature['geometry']['coordinates'][0]  # Assuming the coordinates are in the first list
    
    # Calculate the centroid manually since Python does not have direct access like JS
    centroid_lon = sum([coord[0] for coord in coords]) / len(coords)
    centroid_lat = sum([coord[1] for coord in coords]) / len(coords)

    # Populate data_table with the centroid coordinates and properties
    data_table['Point Longitude'].append(centroid_lon)
    data_table['Point Latitude'].append(centroid_lat)
    data_table['B1'].append(properties.get('B1'))
    data_table['B2'].append(properties.get('B2'))
    data_table['B3'].append(properties.get('B3'))
    data_table['B4'].append(properties.get('B4'))
    data_table['B5'].append(properties.get('B5'))
    data_table['B6'].append(properties.get('B6'))
    data_table['B7'].append(properties.get('B7'))
    data_table['B8'].append(properties.get('B8'))
    data_table['B8A'].append(properties.get('B8A'))
    data_table['B9'].append(properties.get('B9'))
    data_table['B11'].append(properties.get('B11'))
    data_table['B12'].append(properties.get('B12'))

In [757]:
df = pd.DataFrame(data_table)

# Print the DataFrame
print(df)

# Save the result to a CSV file
csv_filename = 'sensrec_statistics.csv'
df.to_csv(csv_filename, index=False)
print(f'Results saved to {csv_filename}')

    Point Longitude  Point Latitude           B1           B2           B3  \
0         30.484469       31.169469   431.389973   495.777345   783.336809   
1         30.520719       31.147858   462.553001   527.157006   847.828152   
2         30.562025       31.126969   408.309305   487.676872   791.746337   
3         30.643553       31.073803   527.390405   590.991075   847.025854   
4         30.730025       31.024497   387.037644   431.183875   751.155225   
5         30.803553       31.009358   459.526337   503.491839   791.586535   
6         30.786636       30.961191   491.265125   535.412679   791.698308   
7         30.797275       30.909275   463.067191   495.029138   783.314697   
8         30.811303       30.900386   462.202349   495.236130   783.291888   
9         30.844330       30.897219   467.367662   503.310207   791.561433   
10        30.872914       30.869497   459.670264   503.574613   775.647977   
11        30.895053       30.852247   507.748035   568.234669   

# calculate the correlation between bands and HM

In [759]:
# read data and display it 
data=pd.read_csv('sensrec_statistics.csv')
data.head()

,Point Longitude,Point Latitude,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12
0,30.484469,31.169469,431.389973,495.777345,783.336809,687.864204,1326.990900,2736.627459,3407.653369,3407.942264,3631.949474,3616.885546,1967.735982,1167.239338
1,30.520719,31.147858,462.553001,527.157006,847.828152,815.659205,1392.231321,2607.975066,3183.611737,3184.147082,3375.413597,3358.266772,2063.342000,1359.297412
2,30.562025,31.126969,408.309305,487.676872,791.746337,727.760201,1335.105696,2607.727379,3088.267017,3151.758948,3312.046855,3311.666720,1959.399558,1223.569170
3,30.643553,31.073803,527.390405,590.991075,847.025854,911.252771,1327.408275,2255.994454,2704.112370,2734.723726,2863.823186,2912.318830,1935.435783,1327.070988
4,30.730025,31.024497,387.037644,431.183875,751.155225,623.218777,1295.257526,2479.464367,2895.127833,2958.805908,3120.168727,3122.411622,1775.594437,1039.496316


In [760]:
# add the HM concentration column to df
data['HM']=[
384.9,
318.3,
342.4,
440.8,
397.6,
380.4,
381.2,
414.5,
333.9,
362.8,
366.9,
407.6,
373.5,
382.0,
385.6,
356.8,
293.4,
332.1,
363.2,
429.9,
378.6,
419.2,
399.6,
189.0,
427.1,
429.2,
392.4,
387.6,
437.0,
444.5,
410.6,
453.4,
409.2,
448.0,
376.7,
395.0,
397.7,
347.8,
328.7,
292.1,
222.7,
152.0,
141.5,
287.4,
340.4,
318.1,
309.6,
325.8,
246.8,
231.3,
326.1,
276.7,
340.9]

In [761]:
# split the target(HM) and get correlation between it and features(bands)
target='HM'
#pearson correlation
corr=data.corr()[target]
print(corr)

Point Longitude   -0.457153
Point Latitude     0.234361
B1                -0.048892
B2                -0.150881
B3                -0.320374
B4                -0.192084
B5                -0.370347
B6                -0.267124
B7                -0.215071
B8                -0.250613
B8A               -0.235791
B9                -0.250612
B11               -0.384041
B12               -0.199664
HM                 1.000000
Name: HM, dtype: float64
